In [8]:
import yaml
import json

### LOAD YAML FILE

In [9]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

### LOAD & SAVE JSON FILE

In [10]:
def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)

In [11]:
def save_json(data, path):
    with open(path, 'w') as outfile:
        json.dump(data, outfile)